In [ ]:
import neptune.new as neptune
from neptune.new.types import File
import pandas as pd
import matplotlib.pyplot as plt
import json

def log_dictionary(a: dict, experiment: neptune.run.Run, prefix: str):
    for key, value in a.items():
        experiment[prefix+"/"+key].log(value)

# Initialize the experiment

In [ ]:
exp = neptune.init(project='dalessioluca/gcnv')

In [ ]:
exp["description"] = "I am logging some text"

### log some configuration parameters

In [ ]:
config_files = ["calling_config.json", "denoising_config.json",  "gcnvkernel_version.json"]
for config in config_files:
    with open("./data1/"+config, 'rb') as f:
        log_dictionary(json.load(f), experiment=exp, prefix='config')

### log the posterior (raw data and images)

In [ ]:
posterior_file = './data1/log_q_tau_tk.tsv'
df = pd.read_csv(posterior_file, sep='\t', header=2, nrows=100)
exp['csv/log_q_tau_tk'] = neptune.types.File.as_html(df)

figure, ax = plt.subplots(ncols=3, figsize=(15, 5))
df.plot(y=['VALUE_0', 'VALUE_1'], ax=ax[0])
df.plot(y='VALUE_0', ax=ax[1])
df.plot(y='VALUE_1', ax=ax[2])

exp['static_image'] = neptune.types.File.as_image(figure)
exp['interactive_img'] = neptune.types.File.as_html(figure)

### log a series of variables

In [ ]:
import re

log_file = "GermlineCNVCallerCohortMode-0.log"

with open("./data1/"+log_file, 'r') as f:
    for line in f:
        match = re.search(r'(ELBO:)(\s*.[0-9.]*)', line)        
        if match:
            try:
                val = float(match.group(2))
                #print(val)
                exp["ELBO"].log(val)
            except:
                pass

In [ ]:
exp.stop()